# Table of Contents
 <p><div class="lev2 toc-item"><a href="#Load-geometries-to-analyze" data-toc-modified-id="Load-geometries-to-analyze-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Load geometries to analyze</a></div><div class="lev2 toc-item"><a href="#Analyze-loss/gain-Wildlife-places" data-toc-modified-id="Analyze-loss/gain-Wildlife-places-02"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Analyze loss/gain Wildlife places</a></div><div class="lev2 toc-item"><a href="#analyze-burned-areas" data-toc-modified-id="analyze-burned-areas-03"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>analyze burned areas</a></div><div class="lev2 toc-item"><a href="#analyze-Glad" data-toc-modified-id="analyze-Glad-04"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>analyze Glad</a></div><div class="lev2 toc-item"><a href="#analyze-precipitation" data-toc-modified-id="analyze-precipitation-05"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>analyze precipitation</a></div><div class="lev2 toc-item"><a href="#ndvi" data-toc-modified-id="ndvi-06"><span class="toc-item-num">0.6&nbsp;&nbsp;</span>ndvi</a></div><div class="lev2 toc-item"><a href="#temperatures" data-toc-modified-id="temperatures-07"><span class="toc-item-num">0.7&nbsp;&nbsp;</span>temperatures</a></div><div class="lev2 toc-item"><a href="#species-richness" data-toc-modified-id="species-richness-08"><span class="toc-item-num">0.8&nbsp;&nbsp;</span>species richness</a></div><div class="lev2 toc-item"><a href="#snow-and-ice" data-toc-modified-id="snow-and-ice-09"><span class="toc-item-num">0.9&nbsp;&nbsp;</span>snow and ice</a></div><div class="lev2 toc-item"><a href="#Water-change" data-toc-modified-id="Water-change-010"><span class="toc-item-num">0.10&nbsp;&nbsp;</span>Water change</a></div><div class="lev2 toc-item"><a href="#droughts?" data-toc-modified-id="droughts?-011"><span class="toc-item-num">0.11&nbsp;&nbsp;</span>droughts?</a></div>

In [1]:
import geopandas as gpd
import ee; ee.Initialize()
import json
import pandas as pd  
import numpy as np
import datetime
import json
import requests


## Load geometries to analyze

In [1]:
with open('/Users/alicia/Downloads/location.json') as data:
    dashboards = json.loads(data.read())

In [4]:
[dashboard['geojson']['features'][0].update({"properties": {'slug':dashboard['slug'], 'areaHa':dashboard['areaHa']}}) for dashboard in dashboards['data']['dashboards']]
geometries = [dict(dashboard['geojson']['features'][0]) for dashboard in dashboards['data']['dashboards']]
for idx, item in enumerate(geometries):
    if len(np.array(item['geometry']['coordinates']).shape)>3:
        geometries[idx]['geometry']['coordinates'] = item['geometry']['coordinates'][0]
collection = ee.FeatureCollection(geometries)

##### Pumalin region

In [10]:
with open('/Users/alicia/Downloads/Pumalin_Park.json') as data:
    pumalin = json.loads(data.read())
geometries = [{"type": "Feature", "geometry":pumalin['features'][0]["geometry"],"properties":{'slug':'pumalinPa', 'areaHa':292491}}]
collection = ee.FeatureCollection(geometries)

In [11]:
collection

## Analyze loss/gain Wildlife places

In [29]:
lwp = ee.Image.pixelArea().mask(ee.Image('users/adampain/LWPs_binary_WGS84'))
regReducer = {
    'collection': collection,
    'reducer': ee.Reducer.sum(),
    'scale':90,
    'tileScale': 4
}
RegionslwpA = lwp.reduceRegions(**regReducer).toList(10000).getInfo()
lwpA ={}
for region in RegionslwpA:
    lwpA[region['properties']['slug']]=region['properties']['sum']/10000
print(lwpA)

{'pumalinPa': 290457.91816669283}


In [32]:
wpLossG = ee.ImageCollection('users/Aliciaarenzana/natGeo/lossGainWPN')
regReducer = {
    'collection': collection,
    'reducer': ee.Reducer.sum(),
    'scale':90,
    'tileScale': 4
}
def reduce2Regions(img):
    loss = ee.Image.pixelArea().rename(['loss']).mask(img.select('loss'))
    gain = ee.Image.pixelArea().rename(['gain']).mask(img.select('gain'))

    reduction = loss.addBands(gain).reduceRegions(**regReducer)
    return ee.Feature(None, ee.Image(img).toDictionary().combine(ee.Dictionary({'test': reduction.toList(1000000)})))

imReduced = wpLossG.map(reduce2Regions).toList(1000)
myOutput = imReduced.getInfo()
output=[]
print(myOutput)
for year in myOutput:
    for region in year['properties']['test']:
         output.append({'year':year['properties']['date'], 
                        'WLPbasHa':lwpA[region['properties']['slug']],
                        'regionAreaHa':region['properties']['areaHa'], 
                        'lossHa':region['properties']['loss']/10000,
                        'gainHa':region['properties']['gain']/10000,
                        'index': ((lwpA[region['properties']['slug']] - (region['properties']['loss']/10000) +(region['properties']['gain']/10000))/lwpA[region['properties']['slug']]), 
                        'slug':region['properties']['slug']})

with open('LossGainWLP.json', 'w') as outfile:
    json.dump(output, outfile)

[{'type': 'Feature', 'geometry': None, 'id': '2010', 'properties': {'date': 2010.0, 'test': [{'type': 'Feature', 'geometry': {'geodesic': True, 'type': 'MultiPolygon', 'coordinates': [[[[-72.24875381299995, -42.493722677999926], [-72.24875521099995, -42.49370921299993], [-72.24875381299995, -42.493722677999926], [-72.25298055199994, -42.4943464399999], [-72.25330705899995, -42.494322703999956], [-72.25408495799991, -42.49425582099991], [-72.25473333699995, -42.49428039499992], [-72.25589544299993, -42.49425871199992], [-72.2569920279999, -42.49429398399991], [-72.25805181899995, -42.494422941999915], [-72.25866712399994, -42.49475279299992], [-72.25995899499992, -42.494878298999936], [-72.26055942699992, -42.49467649099995], [-72.26080401099995, -42.49421409599995], [-72.26149435699995, -42.493537326999956], [-72.26219741099993, -42.49291927799993], [-72.26294679199992, -42.49178007799992], [-72.2631178389999, -42.49150057399993], [-72.26338182699993, -42.49071445199991], [-72.26333092

## analyze burned areas

In [6]:
### ASSETS from GEE lets prepare it to what we need
BurnedAreas= ee.ImageCollection('MODIS/006/MCD64A1').select('BurnDate')
### Reducer to group images
imReducer = {
    'reducer':ee.Reducer.lastNonNull(),
    'parallelScale':10
}
### Reducer to regions extractor
regReducer = {
    'collection': collection,
    'reducer': ee.Reducer.frequencyHistogram().unweighted(),
    'scale':500,
    'tileScale': 4
}
#mapable function to extract geometries
def reduce2Regions(img):
    reduction = ee.Image(img).reduceRegions(**regReducer)
    return ee.Feature(None, ee.Image(img).toDictionary().combine(ee.Dictionary({'test':reduction.toList(100)})))

# grouping images by date (this only work if we have DOY as pixels values)
listYimages=[BurnedAreas.filterDate( ee.Date.fromYMD(year, 1, 1), ee.Date.fromYMD((year+1), 12, 31)).reduce(**imReducer) for year in range(2000, datetime.datetime.now().year)]

# once we have the yearly image production we should reduced to the regions per each year
imReduced=ee.List(listYimages).map(reduce2Regions)
myOutput=imReduced.getInfo()

#reshaping the data (this can also be done in earth engine but is a pain in the ass)
output=[]
for position, year in enumerate(myOutput):
    for region in year['properties']['test']: 
        hist= [{'year':2001+position,'isoWeek':i,'value':0, 'slug':region['properties']['slug']} for i in range(1,54)]
        for key, value in region['properties']['histogram'].items(): #this loop will visit every geometry in every year to agg them into weekly weeks
            if key != 'null':
                myDoy=datetime.datetime(2001+position, 1, 1) + datetime.timedelta(days=int(key)-1)
                week = myDoy.isocalendar()[1]
                hist[week-1]['value'] += value

        output.extend(hist)
with open('burnAreas.json', 'w') as outfile:
    json.dump(output, outfile)
    

## analyze Glad

In [22]:
#From the API
api_asset = '2563cd1a-07d5-4621-99a0-c4a241c4f6b0'

def getData(geostore):
    query = """
    select count(year) as value, year, julian_day from Expanded-GLAD-alerts where confidence_text='confirmed' group by year, julian_day
    """
    params = {
        'sql':query,
        'geostore':geostore

    }
    print(geostore)
    r = requests.get('http://api.resourcewatch.org/v1/query/{0}'.format(api_asset),params=params)
    print(r.text)
    if r.status_code == 200:
        return r.json()['data']

def createGeostore(feature):
    headers={
        "Content-Type":"application/json"
    }
    params = {
        "geojson": {
      "type":"FeatureCollection",
      "features":[feature]}

    }

    r = requests.post('https://api.resourcewatch.org/v1/geostore',data=json.dumps(params), headers=headers)
    if r.status_code == 200:
        return r.json()['data']['id']
    else:
            return [{}]
output=[]    
for region in geometries:
    geostore=createGeostore(region)
    hist= [{'year':year,'isoWeek':i,'value':0, 'slug':region['properties']['slug']} for year in range(2015,2019) for i in range(1,54)]
    data = getData(geostore)
    for row in data: #this loop will visit every year to agg them into weekly weeks
        myDoy=datetime.datetime(row['year'], 1, 1) + datetime.timedelta(days=row['julian_day']-1)
        week = myDoy.isocalendar()[1]
        hist[((row['year']-2014)*week)-1]['value'] += row['value']

    output.extend(hist)

with open('gladAlerts.json', 'w') as outfile:
    json.dump(output, outfile)

82b81a115de755a369718a2b45952e47
Unexpected error


In [ ]:
### From Gee
### ASSETS from GEE lets prepare it to what we need
Glad2017= ee.ImageCollection('projects/glad/alert/UpdResult').select(['alertDate17']).filterDate( ee.Date.fromYMD(2018, 1, 1), ee.Date.fromYMD(2018, 6, 30)).mosaic()
Glad2018= ee.ImageCollection('projects/glad/alert/UpdResult').select(['alertDate18']).mosaic()

### Reducer to regions extractor
regReducer = {
    'collection': collection,
    'reducer': ee.Reducer.frequencyHistogram().unweighted(),
    'scale':500,
    'tileScale': 4
}
#mapable function to extract geometries
def reduce2Regions(img):
    reduction = ee.Image(img).reduceRegions(**regReducer)
    return ee.Feature(None, ee.Image(img).toDictionary().combine(ee.Dictionary({'test':reduction.toList(100)})))

# grouping images by date (this only work if we have DOY as pixels values)
listYimages=[Glad2017, Glad2018]

# once we have the yearly image production we should reduced to the regions per each year
imReduced=ee.List(listYimages).map(reduce2Regions)
myOutput=imReduced.getInfo()

#reshaping the data (this can also be done in earth engine but is a pain in the ass)
output=[]
for position, year in enumerate(myOutput):
    for region in year['properties']['test']: 
        hist= [{'year':2017+position,'isoWeek':i,'value':0, 'slug':region['properties']['slug']} for i in range(1,53)]
        for key, value in region['properties']['histogram'].items(): #this loop will visit every geometry in every year to agg them into weekly weeks
            if key != 'null':
                myDoy=datetime.datetime(2018, 1, 1) + datetime.timedelta(days=int(key)-1)
                week = myDoy.isocalendar()[1]
                hist[week-1]['value'] += value

        output.extend(hist)
with open('Glad.json', 'w') as outfile:
    json.dump(output, outfile)

## analyze precipitation

In [26]:
### ASSETS from GEE lets prepare it to what we need
chirps= ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
### Reducer to group images
imReducer = {
    'reducer':ee.Reducer.sum(),
    'parallelScale':100
}
### Reducer to regions extractor
regReducer = {
    'collection': collection,
    'reducer': ee.Reducer.mean(),
    'scale':5600,
    'tileScale': 16
}
#mapable function to extract geometries
def reduce2Regions(img):
    reduction = ee.Image(img).reduceRegions(**regReducer)
    return ee.Feature(None, ee.Image(img).toDictionary().combine(ee.Dictionary({'test':reduction.toList(10)})))
## manage dates
def weekCalc(myTime):
    x=(8-myTime.isocalendar()[2])
    newDate = myTime + datetime.timedelta(days=x)
    return {'week': newDate.isocalendar()[1],'year':newDate.isocalendar()[0], 'month':newDate.month, 'epoch':newDate.timestamp()*1000}
# grouping images by date (for precipitation lets see the cummulative precipitation per week)
#date management: we need a date management that can reduce every day of the year to its correspondent week + year. we are going to use this to reduce the size of the image to something more...

output=[]
for year in range(1981, 2019):
    if year == 2018:
        endDate = datetime.datetime.fromtimestamp(1535673600)
    else:
        endDate = datetime.datetime(year+1, 1, 1)
    #mDate = datetime.datetime.fromtimestamp(347155200)
    mDate = datetime.datetime(year, 1, 1)
    listDates=[]
    while mDate < endDate:
        dateW= weekCalc(mDate)
        mDate =datetime.datetime.fromtimestamp(dateW['epoch']/1000)
        listDates.append(dateW)
    
    listYimages=[chirps.filterDate(ee.Date(listDates[ix]['epoch']), ee.Date(listDates[ix+1]['epoch'])).reduce(**imReducer).setMulti({'year':date['year'],'week':date['week'], 'month':date['month']}) for ix, date in enumerate(listDates) if ix < (len(listDates)-1)]

    print(year, len(listYimages))
# once we have the yearly image production we should reduced to the regions per each year

    imReduced=ee.List(listYimages).map(reduce2Regions)
    myOutput = imReduced.getInfo()
    for position, weekY in enumerate(myOutput):
        for region in weekY['properties']['test']: 
            output.append({'year': weekY['properties']['year'],'isoWeek': weekY['properties']['week'],'value': region['properties']['mean'], 'slug':region['properties']['slug']})

with open('chirps_weekly-2017.json', 'w') as outfile:
    json.dump(output, outfile)

1981 52
1982 52
1983 52
1984 53
1985 52
1986 52
1987 52
1988 52
1989 52
1990 52
1991 52
1992 52
1993 52
1994 52
1995 52
1996 52
1997 52
1998 52
1999 52
2000 52
2001 52
2002 52
2003 52
2004 52
2005 52
2006 52
2007 52
2008 52
2009 52
2010 52
2011 52
2012 53
2013 52
2014 52
2015 52
2016 52
2017 52
2018 34


## ndvi

In [27]:
### ASSETS from GEE lets prepare it to what we need
ndvi= ee.ImageCollection('MODIS/MOD09GA_NDVI').select(0)
### Reducer to group images
imReducer = {
    'reducer':ee.Reducer.mean(),
    'parallelScale':50
}
### Reducer to regions extractor
regReducer = {
    'collection': collection,
    'reducer': ee.Reducer.mean(),
    'scale':5600,
    'crs':'EPSG:4326',
    'tileScale': 16
}
#mapable function to extract geometries
def reduce2Regions(img):
    reduction = ee.Image(img).reduceRegions(**regReducer)
    return ee.Feature(None, ee.Image(img).toDictionary().combine(ee.Dictionary({'test':reduction.toList(10)})))
## manage dates
def weekCalc(myTime):
    x=(8-myTime.isocalendar()[2])
    newDate = myTime + datetime.timedelta(days=x)
    return {'week': newDate.isocalendar()[1],'year':newDate.isocalendar()[0], 'month':newDate.month, 'epoch':newDate.timestamp()*1000}
# grouping images by date (for precipitation lets see the cummulative precipitation per week)
#date management: we need a date management that can reduce every day of the year to its correspondent week + year. we are going to use this to reduce the size of the image to something more...

output=[]
## fill the numbers with your selected years
for year in range(2005, 2018):
    if year == 2017:
        endDate = datetime.datetime.fromtimestamp(1490832000)
    else:
        endDate = datetime.datetime(year+1, 1, 1)
    mDate = datetime.datetime(year, 1, 1)
    listDates=[]
    while mDate < endDate:
        dateW= weekCalc(mDate)
        mDate =datetime.datetime.fromtimestamp(dateW['epoch']/1000)
        listDates.append(dateW)
    
    listYimages=[ndvi.filterDate(ee.Date(listDates[ix]['epoch']), ee.Date(listDates[ix+1]['epoch'])).reduce(**imReducer).setMulti({'year':date['year'],'week':date['week'], 'month':date['month']}) for ix, date in enumerate(listDates) if ix < (len(listDates)-1)]

    print(year, len(listYimages))
# once we have the yearly image production we should reduced to the regions per each year

    imReduced=ee.List(listYimages).map(reduce2Regions)
    myOutput = imReduced.getInfo()
    for position, weekY in enumerate(myOutput):
        for region in weekY['properties']['test']: 
            output.append({'year': weekY['properties']['year'],'isoWeek': weekY['properties']['week'],'value': region['properties']['mean'] if 'mean' in region['properties'] else None, 'slug':region['properties']['slug']})

with open('ndvi_weekly-2017.json', 'w') as outfile:
    json.dump(output, outfile)

2005 52
2006 52
2007 52
2008 52
2009 52
2010 52
2011 52
2012 53
2013 52
2014 52
2015 52
2016 52
2017 13


## temperatures

## species richness

## snow and ice

## Water change

## droughts?